Importing libraries that we will use

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import os
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import  Dense
import shutil

Then we creating our model and adding to it our layers, which we will train

In [8]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
# Creating our Sequential model
model=Sequential()
for layer in vgg16_model.layers[:-1]:
    # Adding all layers, that VGG16 model already have
    model.add(layer)
for layer in model.layers:
    # Setting them to non_trainable
    layer.trainable= False
# Adding 2 units to our Model, with ship and without ship
model.add(Dense(units=2,activation='softmax'))

Now we need to prepare data for training

In [9]:
train_path = os.getcwd()+'/data/train'
valid_path = os.getcwd()+'/data/valid'
test_path = os.getcwd()+'/data/test'
# print(os.listdir())
# Converting them to format which receives the Keras model
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['not_ships', 'ships'], batch_size=2)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['not_ships', 'ships'], batch_size=2)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
.flow_from_directory(directory=test_path, target_size=(224,224), classes=['not_ships', 'ships'], batch_size=2, shuffle=False)

Found 130 images belonging to 2 classes.
Found 90 images belonging to 2 classes.
Found 114 images belonging to 2 classes.


Once we have prepared data, we can start training our model, but before this, we need to compile our model

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=train_batches,validation_data=valid_batches,epochs=60,verbose=2)

Epoch 1/60
65/65 - 20s - loss: 0.8970 - accuracy: 0.4846 - val_loss: 0.6408 - val_accuracy: 0.6333 - 20s/epoch - 309ms/step
Epoch 2/60
65/65 - 20s - loss: 0.5141 - accuracy: 0.7308 - val_loss: 0.4602 - val_accuracy: 0.7556 - 20s/epoch - 312ms/step
Epoch 3/60
65/65 - 23s - loss: 0.3809 - accuracy: 0.8154 - val_loss: 0.4347 - val_accuracy: 0.7889 - 23s/epoch - 349ms/step
Epoch 4/60
65/65 - 24s - loss: 0.3084 - accuracy: 0.8923 - val_loss: 0.4141 - val_accuracy: 0.8000 - 24s/epoch - 364ms/step
Epoch 5/60
65/65 - 22s - loss: 0.2777 - accuracy: 0.9154 - val_loss: 0.3398 - val_accuracy: 0.8556 - 22s/epoch - 340ms/step
Epoch 6/60
65/65 - 23s - loss: 0.2267 - accuracy: 0.9308 - val_loss: 0.3251 - val_accuracy: 0.8778 - 23s/epoch - 358ms/step
Epoch 7/60
65/65 - 23s - loss: 0.2019 - accuracy: 0.9385 - val_loss: 0.3465 - val_accuracy: 0.8333 - 23s/epoch - 359ms/step
Epoch 8/60
65/65 - 24s - loss: 0.1882 - accuracy: 0.9538 - val_loss: 0.3284 - val_accuracy: 0.8556 - 24s/epoch - 376ms/step
Epoch 9/

After our model got trained we can perdict our images, but first we need to clear our directory

In [11]:
files = glob.glob(os.getcwd()+'/models_predictions/with/*')
for f in files:
    os.remove(f)
files = glob.glob(os.getcwd()+'/models_predictions/without/*')
for f in files:
    os.remove(f)

Now we can set our model task to predict images from our test directory but also we need to get images from our directory

In [15]:
path=os.getcwd()+'/data/test/'
image_paths = []
""" My test images named 'in81.jpg' to 'in137jpg' and 'not81' to 'not137', so that we can easily see what images our model has put correctly and what not.
 This filenames I append to array 'image_paths' """
for i in range(81,138):
    image_paths.append(path+'in'+str(i)+'.jpg')
for i in range(81,138):
    if(i>=101):
        i+=300
    image_paths.append(path+'not'+str(i)+'.jpg')
for image_file in image_paths:
    # Creating from path files to format which receives Keras model
    img_path = image_file
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    # Giving to func array of images, verbose parameter responsible for show results of each epoch, 0 means nothing to show
    predictions=model.predict(x=img_array,verbose=0)
    #  If the first value of 'prediction_array' is lesser thet second,
    # it means that model thinks that this is image without ship and copy this file to folder without  
    if predictions[0][0]>predictions[0][1]:
        print("Image withouth ship")
        shutil.copy(img_path, os.getcwd()+'/models_predictions/without')
    else:
        print("Image with ship")
        shutil.copy(img_path, os.getcwd()+"/models_predictions/with")
    try:
        predictions.any()
    except NameError:
        print("Any images were found")

Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image withouth ship
Image withouth ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image withouth ship
Image withouth ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image withouth ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image withouth ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image with ship
Image withouth ship
Image with ship
Image withouth ship
Image with ship
Image with ship
Image withouth ship
Image withouth ship
Image withouth s

So, now we have our images in directory 'models_predictions' and we can see, how many images our model has predicted correctly